# **Importing tensorflow library**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# **Importing dataset and splitting into train and validation**

In [47]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/tomatoleaf/tomato/train',
    seed=123,
    image_size=(128,128),
    batch_size=32,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/tomatoleaf/tomato/val',
    seed=123,
    image_size=(128,128),
    batch_size=32
)

Found 10000 files belonging to 10 classes.
Found 1000 files belonging to 10 classes.


# **Getting details of training and validation set**

In [48]:
print("Train class names:", train_ds.class_names)
print("Number of training batches:", len(train_ds))
print("Number of validation batches:", len(val_ds))

for images, labels in train_ds.take(1):
    print("First train batch shape:", images.shape)


Train class names: ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']
Number of training batches: 313
Number of validation batches: 32
First train batch shape: (32, 128, 128, 3)


# **Getting number of classes**

In [49]:
num_classes = len(train_ds.class_names)
print(num_classes)

10


# **Defining data augementation layer**

In [50]:
data_augementation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# **Defining CNN model**

In [60]:
model = models.Sequential([
    data_augementation,
    layers.Rescaling(1./255),
    layers.Conv2D(32,(3,3), activation='relu',input_shape=(128,128,3)),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.Conv2D(128,(3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(num_classes,activation='softmax'),
])

# **Compiling model**

In [61]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# **Defining reducing learning rate**

In [62]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6,
)

# **Defining early stopping**

In [63]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True,
)

# **Saving model checkpoints**

In [64]:
checkpoints = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True
)

# **Defining callbacks**

In [65]:
callbacks = [reduce_lr, early_stopping, checkpoints]

# **Training model**

In [66]:
history = model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=callbacks)

Epoch 1/50
311/313 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.2571 - loss: 2.0481
Epoch 1: val_loss improved from inf to 1.09523, saving model to best_model.h5
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 37ms/step - accuracy: 0.2582 - loss: 2.0452 - val_accuracy: 0.6310 - val_loss: 1.0952 - learning_rate: 0.0010
Epoch 2/50
310/313 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5874 - loss: 1.2036
Epoch 2: val_loss improved from 1.09523 to 0.68255, saving model to best_model.h5
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.5878 - loss: 1.2024 - val_accuracy: 0.7830 - val_loss: 0.6826 - learning_rate: 0.0010
Epoch 3/50
310/313 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6787 - loss: 0.9284
Epoch 3: val_loss did not improve from 0.68255
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.6789 - loss: 0.9280 - val_accuracy: 0.7460 - val_loss: 0.7643 - learning_rate: 0.0010
Epoch 4/50
310/313 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7096 - loss: 0.8323
Epoch 4: val_loss 

# **Checking loss and accuracy**

In [67]:
val_loss, val_acc = model.evaluate(val_ds)
print(val_loss)
print(val_acc)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8861 - loss: 0.3346
0.31743961572647095
0.9010000228881836


# **Saving model**

In [68]:
model.save("final_model.h5")

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [69]:
model.save("final_model.keras")